# Everything You Need to Know About Python Environment Variables

## Introduction

Environment variables are powerful mechanisms for storing configuration details of your development project flexibly and securely. They store key-value pairs of data accessible by programming languages such as Python and keep that information hidden from outside parties. In this tutorial, you will learn all must-know concepts and techniques to create and manage environment variables in Python.

## What are environment variables in general?

## Why are Python environment variables important?

## How to retrieve environment variables in Python `os` module?

In [20]:
import os

os.getenv("USER")

'bexgboost'

In [22]:
os.getenv("MYDATABASE")

In [35]:
password = os.getenv("COMPUTER_PASSWORD", "123")

print(password)

123


In [36]:
current_directory = os.environ.get("PWD", None)
current_directory

'/home/bexgboost/articles/2024/4_april'

In [6]:
import os

current_directory = os.environ["PWD"]
current_directory

'/home/bexgboost/articles/2024/4_april'

In [28]:
os.getenv("HOME")

'/home/bexgboost'

In [30]:
os.getenv("CONDA_EXE")

'/home/bexgboost/anaconda3/bin/conda'

In [32]:
os.getenv("PATH")[:46]

'/home/bexgboost/.local/bin:/home/bexgboost/bin'

In [34]:
def count_path_items():
    items = os.getenv("PATH").split(":")

    return len(items)


count_path_items()

79

## Using `python-dotenv` library to manage environment variables in Python effectively

### Setting custom environment variables

```shell
$ touch ~/.env
```

```shell
CUSTOM_API_LINK=https://myapi.com/v1/api
SNOWFLAKE_USERNAME=bexgboost
MYSQL_DATABASE_PASSWORD=as3ndf03
```

```bash
LONG_ENV="This is an environment
variable with a long message"
```

```bash
LONG_ENV="This is an \"environment
variable with a long message"
```

```shell
# You can also add comments anywhere with #
CUSTOM_API_LINK=https://myapi.com/v1/api
SNOWFLAKE_USERNAME=bexgboost
MYSQL_DATABASE_PASSWORD=as3ndf03
```

```shell
FIRST_NAME=John
LAST_NAME=Doe
FULL_NAME="I am ${FIRST_NAME} ${LAST_NAME}"
```

### Retrieving custom environment variables with `python-dotenv`

In [8]:
from dotenv import load_dotenv

load_dotenv()

True

In [7]:
import os

os.getenv("CUSTOM_API_LINK")

'https://myapi.com/v1/api'

In [3]:
os.getenv("SNOWFLAKE_USERNAME")

'bexgboost'

In [5]:
load_dotenv("/home/bexgboost/.env")

True

### Working with .env files in Jupyter

In [1]:
%load_ext dotenv
%dotenv

In [4]:
import os

os.getenv("SNOWFLAKE_USERNAME")

'bexgboost'

## Best practices for using environment variables in Python

```
$ touch .gitignore
$ echo ".env" >> .gitignore
```

- Use descriptive naming
- Scope control, set variables only within a specific environment or project. Don't add project-specific passwords or configs to your global .env file
- Always use .envs
- Always make sure that .envs are in gitignore
- Create multiple .env files for different parts of development
- Then, add them as .env*

## Conclusion